# Android Multiple Workloads - complex example

This complex example shows multiple workloads being executed in multiple configurations.

Please check the notebooks in **examples/android/benchmarks/** and **examples/android/workloads/** to get more details on each of the possible workloads and how you can visualise their results.

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2016-12-09 17:56:46,425 INFO    : root         : Using LISA logging configuration:
2016-12-09 17:56:46,426 INFO    : root         :   /home/vagrant/lisa/logging.conf


In [2]:
%pylab inline

import collections
import copy
import json
import os
from time import sleep

# Support to access the remote target
import devlib
from env import TestEnv

# from devlib.utils.android import adb_command

# Import support for Android devices
from android import Screen, Workload, System

# Support for trace events analysis
from trace import Trace

# Suport for FTrace events parsing and visualization
import trappy

import datetime

Populating the interactive namespace from numpy and matplotlib


## Support Functions

In [3]:
def set_performance():
    target.cpufreq.set_all_governors('performance')

def set_powersave():
    target.cpufreq.set_all_governors('powersave')

def set_interactive():
    target.cpufreq.set_all_governors('interactive')

def set_sched():
    target.cpufreq.set_all_governors('sched')

def set_ondemand():
    target.cpufreq.set_all_governors('ondemand')
    
    for cpu in target.list_online_cpus():
        tunables = target.cpufreq.get_governor_tunables(cpu)
        target.cpufreq.set_governor_tunables(
            cpu,
            'ondemand',
            **{'sampling_rate' : tunables['sampling_rate_min']}
        )

In [4]:
def experiment(wl, res_dir, conf_name, wload_name, collect=''):
    
    ##################################
    # Initial setup

    # Load workload params
    wload_kind = wload_name.split()[0]
    iterations = int(wload_name.split()[1])
    wload_tag = wload_name.split()[2]\
                .replace('https://youtu.be/', '')\
                .replace('?t=', '_')
            
    # Check for workload being available
    wload = Workload.get(te, wload_kind)
    if not wload:
        return {}
    
    # Setup test results folder
    exp_dir = os.path.join(res_dir, conf_name, "{}_{}".format(wload_kind, wload_tag))
    os.system('mkdir -p {}'.format(exp_dir));

    # Configure governor
    confs[conf_name]['set']()
    
    # Unlock device screen (assume no password required)
    target.execute('input keyevent 82')
    # Configure screen to max brightness and no dimming
    Screen.set_brightness(target, percent=100)
    Screen.set_dim(target, auto=False)
    Screen.set_timeout(target, 60*60*10) # 10 hours should be enought for an experiment
    
    ####################################
    # Start the required tracing command
    
    if 'ftrace' in collect:
        # Start FTrace and Energy monitoring
        te.ftrace.start()
    elif 'systrace' in collect:
        # Get the systrace time
        match = re.search(r'systrace_([0-9]+)', collect)
        if match:
            systrace_time = match.group(1)
        else:
            logging.warning("Systrace time NOT defined, tracing for 10[s]")
            systrace_time = 10
        # Start systrace
        trace_file = os.path.join(te.res_dir, 'trace.html')
        systrace_output = System.systrace_start(te, trace_file, systrace_time)
    
    
    ###########################
    # Run the required workload
    
    # Jankbench
    if 'Jankbench' in wload_name:
        db_file, nrg_report = wload.run(exp_dir, wload_tag, iterations, collect)

    # UiBench
    elif 'UiBench' in wload_name:
        test_name = wload_name.split()[2]
        duration_s = int(wload_name.split()[3])
        db_file, nrg_report = wload.run(exp_dir, test_name, duration_s)

    # YouTube
    elif 'YouTube' in wload_name:
        video_url = wload_name.split()[2]
        video_duration_s = int(wload_name.split()[3])
        db_file, nrg_report = wload.run(exp_dir, video_url, video_duration_s)

    # RTApp based workloads
    elif 'RTApp' in wload_name:
        rtapp_kind = wload_name.replace('RTApp ', '')
        db_file, nrg_report = rtapp_run(rtapp_kind)

        
    ###########################
    # Reset and return results

    # Stop the required trace command
    if 'ftrace' in collect:
        te.ftrace.stop()
        # Collect and keep track of the trace
        trace_file = os.path.join(exp_dir, 'trace.dat')
        te.ftrace.get_trace(trace_file)
    elif 'systrace' in collect:
        if systrace_output:
            logging.info('Waiting systrace report [%s]...', trace_file)
            systrace_output.wait()
        else:
            logging.warning('Systrace is not running!')

    # Reset screen brightness and auto dimming
    Screen.set_defaults(target)
        
    # Dump platform descriptor
    te.platform_dump(exp_dir)

    # return all the experiment data
    if 'trace' in collect:
        return {
            'dir'             : exp_dir,
            'db_file'         : db_file,
            'nrg_report'      : nrg_report,
            'trace_file'      : trace_file,
        }
    else:
        return {
            'dir'             : exp_dir,
            'db_file'         : db_file,
            'nrg_report'      : nrg_report,
            'nrg_file'        : nrg_file,
        }

In [5]:
def run_experiments(test_confs, wloads, confs, verbose=False):

    # Make sure we have a list of configuraitons to test
    if not isinstance(test_confs, list):
        test_confs = [test_confs]

    # Intialize Workloads for this test environment
    wl = Workload(te)

    # Change to info once the notebook runs ok
    if verbose:
        LisaLogging.setup(level=logging.DEBUG)
    else:
        LisaLogging.setup(level=logging.INFO)

    # The set of results for each comparison test
    results = collections.defaultdict(dict)

    # Run the benchmark in all the configured configurations
    for conf_name in test_confs:

        # Setup data to be collected
        try:
            collect = confs[conf_name]['collect']
            logging.info("Enabling collection of: %s", collect)
        except:
            collect = ''

        # Enable energy collection only if an emeter has been configured
        if 'energy' in collect:
            if 'emeter' not in my_conf or not te.emeter:
                logging.warning('Disabling ENERGY collection')
                logging.info('EMeter not configured or not available')
                collect = collect.replace('energy', '')
            else:
                logging.debug('Enabling ENERGY collection')

        # Run each workload
        idx = 0
        for wload_name in wloads:
            
            # Skip workload if not enabled by the configuration
            try:
                enabled = False
                enabled_workloads = confs[conf_name]['wloads']
                for wload in enabled_workloads:
                    if wload in wload_name:
                        enabled = True
                        break
                if not enabled:
                    logging.debug('Workload [%s] disabled',
                                 wload_name)
                    continue
            except:
                # No workload filters defined, execute all workloads
                logging.debug('All workloads enabled')
                pass

            # Log test being executed
            idx = idx + 1
            wload_kind = wload_name.split()[0]
            logging.info('------------------------')
            logging.info('Test %d: %s in %s configuration',
                         idx, wload_kind.upper(), conf_name.upper())
            logging.info('        %s', wload_name)
            
            res = experiment(wl, te.res_dir, conf_name, wload_name, collect)
            results[conf_name][wload_name] = copy.deepcopy(res)

        # Save collected results
        res_file = os.path.join(te.res_dir, conf_name, 'results.json')
        with open(res_file, 'w') as fh:
            json.dump(results[conf_name], fh, indent=4)

## Test environment setup
For more details on this please check out **examples/utils/testenv_example.ipynb**.

**devlib** requires the ANDROID_HOME environment variable configured to point to your local installation of the Android SDK. If you have not this variable configured in the shell used to start the notebook server, you need to run a cell to define where your Android SDK is installed or specify the ANDROID_HOME in your target configuration.

In case more than one Android device are conencted to the host, you must specify the ID of the device you want to target in **my_target_conf**. Run **adb devices** on your host to get the ID.

In [6]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"      : 'android',
    "device"        : "HT6670300102",
    "ANDROID_HOME"  : '/home/vagrant/lisa/tools/android-sdk-linux/',

    # Folder where all the results will be collected
    "results_dir" : "Android_Multiple_Workloads",

    # Define devlib modules to load
    "modules"     : [
        'cpufreq'       # enable CPUFreq support
    ],

    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_overutilized",
            "sched_contrib_scale_f",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_tune_tasks_update",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_energy_diff",
            "cpu_frequency",
            "cpu_idle",
            "cpu_capacity",
         ],
         "buffsize" : 10 * 1024,
    },

    # Tools required by the experiments
    "tools"   : [ 'trace-cmd' ],
}

In [7]:
# List of possible workloads to run, each workload consists of a workload name
# followed by a list of workload specific parameters
test_wloads = [
# YouTube workload:
# Params:
# - iterations: number of read/write operations to execute
# - URL:        link to the video to use (with optional start time)
# - duration:   playback time in [s]
    'YouTube 1 https://youtu.be/XSGBVzeBUbk?t=45s   60',

# Jankbench workload:
# Params:
# - iterations: number of read/write operations to execute
# - id:         benchmakr to run
    'Jankbench 1 list_view',
    'Jankbench 1 image_list_view',
    'Jankbench 1 shadow_grid',
    'Jankbench 1 low_hitrate_text',
    'Jankbench 1 high_hitrate_text',
    'Jankbench 1 edit_text',
    
    # Multi iterations
    'Jankbench 3 list_view',
    'Jankbench 3 image_list_view',
    'Jankbench 3 shadow_grid',
    'Jankbench 3 low_hitrate_text',
    'Jankbench 3 high_hitrate_text',
    'Jankbench 3 edit_text',

# UiBench workload:
# Params:
# - test_name:  The name of the test to start
# - duration:   playback time in [s]
    'UiBench 1 TrivialAnimation 10',

# RT-App workload:
# Params:
# - configration: tasks configuration to run
# - [configuration specific parameters]
    'RTApp STAccount 6',
    'RTApp RAMP',
]

In [8]:
# Available test configurations
# 'set' : a setup function to be called before starting the test
# 'collect' defines what we want to collect as a list of strings.
#     Supported values are
#        energy   - Use the my_conf's defined emeter to measure energy consumption across experiments
#        ftrace   - Collect an execution trace using trace-cmd
#        systrace - Collect an execution trace using Systrace/Atrace
#     NOTE: energy is automatically enabled in case an "emeter" configuration is defined in my_conf

confs = {
     'std' : {
         'set'     : set_interactive,
         'wloads'  : ['Jankbench 1 list_view'],
         'collect' : 'ftrace',
     },
     'eas' : {
        'set'      :  set_sched,
        'wloads'   : ['Jankbench 1 list_view'],
        'collect'  : 'ftrace',
     }
}

In [9]:
# List of experiments to run
experiments = ['std', 'eas']

In [10]:
# Initialize a test environment using:
te = TestEnv(my_conf, wipe=False)
target = te.target

2016-12-09 17:56:54,280 INFO    : TestEnv      : Using base path: /home/vagrant/lisa
2016-12-09 17:56:54,282 INFO    : TestEnv      : Loading custom (inline) target configuration
2016-12-09 17:56:54,283 INFO    : TestEnv      : External tools using:
2016-12-09 17:56:54,284 INFO    : TestEnv      :    ANDROID_HOME: /home/vagrant/lisa/tools/android-sdk-linux/
2016-12-09 17:56:54,284 INFO    : TestEnv      :    CATAPULT_HOME: /home/vagrant/lisa/tools/catapult
2016-12-09 17:56:54,284 INFO    : TestEnv      : Devlib modules to load: ['cpufreq']
2016-12-09 17:56:54,285 INFO    : TestEnv      : Connecting Android target [HT6670300102]
2016-12-09 17:56:54,285 INFO    : TestEnv      : Connection settings:
2016-12-09 17:56:54,286 INFO    : TestEnv      :    {'device': 'HT6670300102'}
2016-12-09 17:56:54,468 INFO    : android      : ls command is set to ls -1
2016-12-09 17:56:55,511 INFO    : TestEnv      : Initializing target workdir:
2016-12-09 17:56:55,513 INFO    : TestEnv      :    /data/loc

## Workloads Execution and Data Collection

In [11]:
run_experiments(experiments, test_wloads, confs, True)

2016-12-09 17:57:12,090 INFO    : Workload     : Workloads available on target:
2016-12-09 17:57:12,093 INFO    : Workload     :   ['YouTube', 'Jankbench', 'UiBench']
2016-12-09 17:57:12,098 INFO    : root         : Using LISA logging configuration:
2016-12-09 17:57:12,099 INFO    : root         :   /home/vagrant/lisa/logging.conf
2016-12-09 17:57:12,101 INFO    : root         : Enabling collection of: ftrace
2016-12-09 17:57:12,102 DEBUG   : root         : Workload [YouTube 1 https://youtu.be/XSGBVzeBUbk?t=45s   60] disabled
2016-12-09 17:57:12,103 INFO    : root         : ------------------------
2016-12-09 17:57:12,104 INFO    : root         : Test 1: JANKBENCH in STD configuration
2016-12-09 17:57:12,104 INFO    : root         :         Jankbench 1 list_view
2016-12-09 17:57:12,105 DEBUG   : Jankbench    : Workload created
2016-12-09 17:57:13,628 INFO    : Screen       : Set brightness: 100%
2016-12-09 17:57:14,052 INFO    : Screen       : Dim screen mode: OFF
2016-12-09 17:57:14,4

In [14]:
!tree {te.res_dir}

/home/vagrant/lisa/results/Android_Multiple_Workloads
├── eas
│   ├── Jankbench_list_view
│   │   ├── BenchmarkResults
│   │   ├── platform.json
│   │   └── trace.dat
│   └── results.json
└── std
    ├── Jankbench_list_view
    │   ├── BenchmarkResults
    │   ├── platform.json
    │   └── trace.dat
    └── results.json

4 directories, 8 files
